<a href="https://colab.research.google.com/github/SubhamPanda2003/Machine_learning/blob/master/breast_cancer_prediction_using_alh_new_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
#================================================================================================================
#----------------------------------------------------------------------------------------------------------------
#									K NEAREST NEIGHBOURS
#----------------------------------------------------------------------------------------------------------------
#================================================================================================================

# Details of implementation/tutorial is in : http://madhugnadig.com/articles/machine-learning/parallel-processing/2017/02/10/implementing-k-nearest-neighbours-in-parallel-from-scratch.html
from multiprocessing import Process, Queue
import difflib, random, time
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import random
from collections import Counter
from sklearn import preprocessing
from itertools import repeat
import multiprocessing as mp
import time
from sklearn import datasets
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from csv import writer
#for plotting
plt.style.use('ggplot')

class CustomKNN:
	def splitlist(self,inlist, chunksize):
		return [inlist[x:x+chunksize] for x in range(0, len(inlist), chunksize)]
	def __init__(self):
		self.accurate_predictions = 0
		self.total_predictions = 0
		self.accuracy = 0.0

	def predict(self, training_data, to_predict, k=2,arr={}):   
		s = time.perf_counter()                      #-----------------------------------value of k---------------------------------------------------
		# if len(training_data) >= k:
		# 	print("K cannot be smaller than the total voting groups(ie. number of training data points)")
		# 	return
		
		distributions = []
		for group in training_data:
			for features in training_data[group]:
				# Find euclidean distance using the numpy function
				euclidean_distance = np.linalg.norm(np.array(features)- np.array(to_predict))
				distributions.append([euclidean_distance,group, features])
		# Find the k nearest neighbors
		results = [i[1] for i in sorted(distributions)[:k]]
		res = [i[2] for i in sorted(distributions)[:k]]
		# Figure out which is the most common class amongst the neighbors.
		result1 = Counter(results).most_common(1)[0][0]
		# confidence = Counter(results).most_common(1)[0][1]/k
		ir1 = pd.DataFrame();
		b=0;g=0;
		for i in results:
			if(i==0):
	 			b+=1
			else:
				g+=1
		h=g*b
		ir1 = pd.DataFrame(res, columns =[x for x in range(9)]) 
		ir2 = pd.DataFrame.from_dict(results)
		nm=0
		x={}

		for i in to_predict:
			x[nm]=i
			nm+=1
		to_predict1=pd.DataFrame(np.array(to_predict).reshape(1,-1), columns =[x for x in range(9)]) 
		
		# with open('event.csv', 'a') as f_object:
 
    # # Pass this file object to csv.writer()
    # # and get a writer object
		# 	writer_object = writer(f_object)
 
    # # Pass the list as an argument into
    # # the writerow()
		# 	for list in res:
		# 		writer_object.writerow(list)
		# 	writer_object.writerow(results)
		# 	writer_object.writerow(to_predict)
    # # Close the file object
		# 	f_object.close()
		# print(to_predict1)
		# to_predict1= pd.DataFrame(to_predict, columns =[x for x in range(24)])
		# print(type(to_predict1))
# X -> features, y -> label
		X = ir1
		y = ir2
		y = y.values.ravel()
		
		# print(y)
# dividing X, y into train and test data
		# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
		
  
# training a linear SVM classifier
		e = time.perf_counter()
		# print(s," ",e," " ,e-s,"-----")
		# print(res)
		if(h!=0):
			from sklearn.svm import SVC
			svm_model_linear = SVC(kernel = 'linear').fit(X, y)
			# print(11)
			result = svm_model_linear.predict(to_predict1)
			# print(result)
			# 		print(distributions)
			return result, to_predict
		else:
			return result1,to_predict

	
	
	def test(self, test_set, training_set):
		pool = mp.Pool(processes= 1)     #---------------------------------------------no of processs---------------------------------------------------------------------------------

		arr = {}
		for ii in range(36,37):
		
		# # Where the magic happens, this is where we parallelize our code. While testing for the classes of incoming points,
		# # we divide the incoming data points and feed them into the predict funtion in parallel.
		# # I have used the starpmap function of multiprocessing library for this purpose. 
		# # The training data gets repeated to get an iterable of the training dataset for the map function, ie. the predict funtion, to be applied on.
			for group in test_set:
		# s = time.perf_counter()
			
				arr[group] =  pool.starmap(self.predict, zip(repeat(training_set), test_set[group], repeat(11))) #-----------------------------------value of k---------------------------------------------------
		# e = time.perf_counter()
			# print(s," ",e," " ,e-s)
		#Calculating Accuracy - The accuracy code has to be modified due to the induced parallelism. 
		# It is no longer possible to determinstically calculate the accurate predictions where multiple subprocesses are doing the same increment.
		# pool.close()
		# processes=8
		# mainwordlistsplitted = self.splitlist(training_set, int(len(training_set)/processes))
		# print ("list ready")
		# for submainwordlist in mainwordlistsplitted:
		# 	print ("sub")
		# 	p = Process(target=self.predict, args=(submainwordlist,test_set,201,arr))
		# 	p.Daemon = True
		# 	p.start()
		# for submainwordlist in mainwordlistsplitted:
		# 	p.join()
			for group in test_set:
				for data in test_set[group]:
					for i in arr[group]:
						if data == i[1]:
							self.total_predictions += 1
							if group == i[0]:
								self.accurate_predictions+=1
		
			self.accuracy = 100*(self.accurate_predictions/self.total_predictions)
			print(ii," Acurracy :", str(self.accuracy) + "%")
		

def mod_data(df):
	df.replace('?', 1, inplace = True)
	df.replace('M', 0, inplace = True)
	df.replace('B', 1, inplace = True)
	df.replace('id', 1, inplace = True)


def main():
	df = pd.read_csv(r"./breast-cancer1.csv",header=None)
	mod_data(df)
	
  # X=df.drop(columns=1,axis=1)
	X=df.drop(columns=0,axis=1)
  # X=X.drop(columns=32,axis=1)
	# X=X.drop(0,axis=0)
	x = X[[1,2,3,4,5,6,7,8,9]].values #returns a numpy array
	min_max_scaler = preprocessing.StandardScaler()
	x_scaled = min_max_scaler.fit_transform(x)
	X[[1,2,3,4,5,6,7,8,9]]= pd.DataFrame(x_scaled) #Replace df with normalized values
	dataset = X.astype(float).values.tolist()
	print(X[10].value_counts())
	for i in range(20,30):
	
		# random.shuffle(dataset)

	#20% of the available data will be used for testing
		test_size = 0.2

	#The keys of the dict are the classes that the data is classfied into
		training_set = {2: [], 4:[]}
		test_set = {2: [], 4:[]}
		training_data,test_data=train_test_split(dataset,test_size=0.2,random_state=i)
		# training_data = dataset[:-int(test_size * len(dataset))]
		# test_data = dataset[-int(test_size * len(dataset)):]
	
		# print(training_data)
		# print(test_data)
		print(training_data[0][-1])
		for record in training_data:
			training_set[record[-1]].append(record[:-1]) # Append the list in the dict will all the elements of the record except the class

	#Insert data into the test set
		for record in test_data:
			test_set[record[-1]].append(record[:-1]) # Append the list in the dict will all the elements of the record except the class
	# df = pd.read_csv(r"./letter1.csv")
	# mod_data(df)
	# dataset = df.astype(float).values.tolist()
	# training_set1 = {0: [], 1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[],10:[],11:[],12:[],13:[],14:[],15:[],16:[],17:[],18:[],19:[],20:[],21:[],22:[],23:[],24:[],25:[]}
	# training_data1 = dataset
	# print(type(training_set1))
	# for record in training_data1:
	# 	training_set1[record[-1]].append(record[:-1]) # Append the list in the dict will all the elements of the record except the class
		s = time.perf_counter()
		knn = CustomKNN()
		knn.test(test_set, training_set)
		e = time.perf_counter()
	
		print("Exec Time: ", e-s)
	
if __name__ == "__main__":
	main()

2    458
4    241
Name: 10, dtype: int64
4.0
36  Acurracy : 98.17518248175182%
Exec Time:  1.1717854819999047
2.0
36  Acurracy : 97.59615384615384%
Exec Time:  1.261563733999992
4.0
36  Acurracy : 98.1203007518797%
Exec Time:  1.2043675619997884
2.0
36  Acurracy : 98.03149606299213%
Exec Time:  1.265211415000067
4.0
36  Acurracy : 99.19354838709677%
Exec Time:  1.1920911719998912
2.0
36  Acurracy : 97.77777777777777%
Exec Time:  1.1988125769998987
2.0
36  Acurracy : 96.84684684684684%
Exec Time:  1.1726624200000515
4.0
36  Acurracy : 97.88135593220339%
Exec Time:  1.1622534980001547
2.0
36  Acurracy : 97.88135593220339%
Exec Time:  1.1754580520000673
2.0
36  Acurracy : 99.30555555555556%
Exec Time:  1.1496578070000396
